In [6]:
import torch
import torch.nn.functional as F
! pip install pytorch_lightning
! pip install torchmetrics
import pytorch_lightning as pl
import torchmetrics
import torch
from torch import nn
from torch import optim
from torch.utils.data import random_split
from torchvision import transforms
import torchvision.datasets as datasets
import pytorch_lightning.loggers as pl_loggers
from dataset import wiki_dataset
from dataloader import wiki_dataloader
num_classes = 10


ModuleNotFoundError: ignored

In [2]:
class LSTM1(nn.Module):
    def __init__(self, n_vocab, 
                       seq_size, 
                       embedding_size, 
                       lstm_size):

        super(LSTM1, self).__init__()
        self.seq_size = seq_size
        self.lstm_size = lstm_size
        self.lstm = nn.LSTM(seq_size, embedding_size,
                            lstm_size,
                            batch_first=True, dropout=0.5)
        #nn.utils.clip_grad_norm_(self.lstm.parameters(), clip) - between backwards and optimizer step

    def forward(self, x, prev_state):
        embed = self.embedding(x)
        output, state = self.lstm(embed, prev_state)
        logits = self.fc(output)

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=1e-2)
        nn.utils.clip_grad_norm_(self.lstm.parameters(), clip)

        return logits, state

In [4]:
    def training_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        #l2_norm = sum(p.pow(2.0).sum() for p in self.parameters()).item()
        #l1_norm = sum(p.abs().sum() for p in model.parameters()).item()
        loss = self.loss(logits, label) # + l2_norm + l1_norm
        tensorboard_logs = {'loss': {'train': loss.detach()}}
        self.log("training loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'val': loss.detach()}}
        self.log("validation loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

    def test_step(self, batch, batch_idx):
        data, label = batch
        logits = self.forward(data)
        loss = self.loss(logits, label)
        tensorboard_logs = {'loss': {'test': loss.detach()}}
        self.log("test loss", loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return {"loss": loss, "log": tensorboard_logs}

if __name__ == '__main__':
    # Load datasets
  train = wiki_dataset('../wiki.train.txt', training=True, token_map='create')
  valid = wiki_dataset('../wiki.valid.txt', training=False, token_map=train.token_map)
  test = wiki_dataset('../wiki.test.txt', training=False, token_map=train.token_map)
  datasets = [train, valid, test]

    # Load dataloader
    dataloader = wiki_dataloader(datasets=datasets, batch_size=20)

    # Make model and train
    model = LSTM1(n_vocab=len(train.unique_tokens), 
                       seq_size=30, 
                       embedding_size=100, 
                       lstm_size=2)
    tb_logger = pl_loggers.TensorBoardLogger("./lightning_logs/", name="ff")
    trainer = pl.Trainer(logger=tb_logger, max_epochs=10)
    trainer.fit(model, dataloader)
    result = trainer.test(model, dataloader)
    print(result)

NameError: ignored